Passo 1 - Extração dos arquivos do Twitter 

Notebook responsável por extrair os dados a partir de uma query configurável e salvar os dados em arquivos para processamento pelos demais passos do Pipeline.

Setup de bibliotecas

In [1]:
from collections import Counter
from operator import itemgetter
import tweepy
import pandas as pd
import openpyxl
import datetime
import matplotlib.pyplot as plt
import matplotlib
import networkx as nx
import re
import os
import dataframe_image as dfi
from wordcloud import WordCloud
from wordcloud import ImageColorGenerator
from wordcloud import STOPWORDS

Configuração de Token de Acesso

In [9]:
client = tweepy.Client(bearer_token='Insira o token de acesso', wait_on_rate_limit=True)

Configuração de arquivos para salvamento dos dados e diretório padrão de armazenamento

In [ ]:
print(os.getcwd())

rotulo = "Nome do Arquivo" # nome do arquivo que será usado como padrão para criação dos arquivos
data_pesquisa = "0101" # data referencia usada para criação dos arquivos
save_path = 'C:\\diretório\\' # diretório padrão para criação dos arquivos

file_dado_raw_original = save_path + "\\" + rotulo + "\\" + rotulo + '_' + data_pesquisa + '_' + 'original.csv'
file_dado_raw = save_path + "\\" + rotulo + "\\" + rotulo + '_CR_' + data_pesquisa + '_raw.csv'
file_edges = save_path + "\\" + rotulo + "\\" + rotulo + '_CR_' + data_pesquisa + '_edges.csv'
file_edges_full = save_path + "\\" + rotulo + "\\" + rotulo + '_CR_' + data_pesquisa + '_edges_full.csv'
file_nodes = save_path + "\\" + rotulo + "\\" + rotulo + '_CR_' + data_pesquisa + '_nodes.csv'
file_hist = save_path + "\\" + rotulo + "\\" + rotulo + '_CR_' + data_pesquisa + '_hist.png'

print("Arquivo Original >" + file_dado_raw_original)
print("Arquivo Original Hidratado >" + file_dado_raw)
print("Arquivo Edges sem Tweets sem menção >" + file_edges)
print("Arquivo Edges com Tweets sem menção >" + file_edges_full)
print("Arquivo Nodes > " + file_nodes)
print("Arquivo Grafico Distr Tweets x tempo >" + file_hist)

Setup de query de extração dos tweets

In [13]:
query = 'Palavra chave para query lang:pt' # CR = com retweets SR = sem retweets
Limite_ratio = 100000 # limite max tweets a ser recuperado 
inicio_time = '2023-01-01 T00:00:00Z' # a partir de que data e hora será baseada a pesquisa

tweets = tweepy.Paginator(client.search_recent_tweets, start_time=inicio_time, query=query,tweet_fields=['id','created_at', 'author_id',  'conversation_id', 'entities', 'geo', 'in_reply_to_user_id', 'lang', 'public_metrics', 'referenced_tweets', 'reply_settings', 'context_annotations','withheld','source', 'text'], max_results=100).flatten(limit=Limite_ratio)


Exemplos de tipos diferentes de queries - Usando data de Inicio e Fim para as pesquisas, buscando dados dos ultimos sete dias ou dados históricos

In [14]:
#inicio_time = '2023-07-24T00:00:00Z'
#fim_time = '2023-07-24T120:00:00Z'

#tweets = tweepy.Paginator(client.search_all_tweets, start_time=inicio_time, end_time=fim_time, query=query,tweet_fields=['id','created_at', 'author_id',  'conversation_id', 'entities', 'geo', 'in_reply_to_user_id', 'lang', 'public_metrics', 'referenced_tweets', 'reply_settings', 'context_annotations','withheld','source', 'text'], max_results=100).flatten(limit=Limite_ratio)
#tweets = tweepy.Paginator(client.search_recent_tweets, start_time=inicio_time, end_time=fim_time, query=query,tweet_fields=['id','created_at', 'author_id',  'conversation_id', 'entities', 'geo', 'in_reply_to_user_id', 'lang', 'public_metrics', 'referenced_tweets', 'reply_settings', 'context_annotations','withheld','source', 'text'], max_results=100).flatten(limit=Limite_ratio)


Preparação dos arquivos a partir do Arquivo de Tweets original

In [ ]:
data2 = [[tweet.created_at, tweet.author_id, tweet.text,tweet.conversation_id, tweet.entities, tweet.geo, tweet.in_reply_to_user_id, tweet.lang, tweet.public_metrics, tweet.referenced_tweets, tweet.reply_settings, tweet.context_annotations, tweet.withheld,tweet.source] for tweet in tweets]
data3 = pd.DataFrame(data2, columns = ["created_at", "author_id", "text", "conversation_id", "entities", "geo", "in_reply_to_user_id", "lang", "public_metrics", "referenced_tweets", "reply_settings", "context_annotations", "withheld","source"]) 
data3.to_csv(file_dado_raw_original)

In [21]:
matplotlib.rcParams['timezone'] = 'America/Sao_Paulo'
data3['created_at'] = pd.to_datetime(data3['created_at'])
data3['Created_at_convert'] = data3['created_at'].dt.tz_convert("America/Sao_Paulo")
data3['temp_entities'] = data3['entities'].astype(str).str.extract("mentions': \[(.*?)\]")
data3 = data3.assign(var1=data3['temp_entities'].str.split("}, {")).explode('var1')
data3['mention_username'] = data3['var1'].astype(str).str.extract("'username': '(\w+)")
data3['mention_userid'] = data3['var1'].astype(str).str.extract("'id': '(\w+)")
data3['mention_username'] = data3['mention_username'].str.lower()
data3 = data3.set_index(['created_at'])
data3.to_csv(file_dado_raw)

Geração de um gráfico de distribuição dos tweets ao longo da janela de tempo de extração

In [22]:

x_hist = data3.hist(column='Created_at_convert',bins=8, grid=False, figsize=(12,8), color='#86bf91', zorder=2, rwidth=0.9)

plt.savefig(file_hist)

Geração do arquivo de Egdes para importação pela ferramenta Gephi

In [32]:
data4 = pd.DataFrame(data3, columns = ["author_id", "mention_username", 'Created_at_convert']) 
data4['Timeset'] = data4['Created_at_convert'].map(lambda x: x.isoformat())
data4.rename(columns = {'author_id':'Source', 'mention_username':'Target'}, inplace = True)
data4.Target = data4.Target.fillna('tweet_sem_menção')
data4.to_csv(file_edges_full)
data4 = data4[data4.Target.notnull()]
data4.to_csv(file_edges)

Geração do arquivo de Nodes para importação pela ferramenta Gephi


In [40]:
Nodes_data_source = pd.DataFrame(data3, columns = ["author_id", "Label", "Tipo", "Created_at_convert"]) 
Nodes_data_source['Timeset'] = Nodes_data_source['Created_at_convert'].map(lambda x: x.isoformat())
Nodes_data_source = Nodes_data_source.drop_duplicates(subset=['author_id'])
Nodes_data_Target = pd.DataFrame(data3, columns = ["mention_username", "Label", "Tipo", "Created_at_convert"]) 
Nodes_data_Target['Timeset'] = Nodes_data_Target['Created_at_convert'].map(lambda x: x.isoformat())
Nodes_data_Target = Nodes_data_Target[Nodes_data_Target.mention_username.notnull()]
Nodes_data_Target = Nodes_data_Target.drop_duplicates(subset=['mention_username'], keep = 'last')
Nodes_data_source.rename(columns = {'author_id':'Id'}, inplace = True)
Nodes_data_Target.rename(columns = {'mention_username':'Id'}, inplace = True)
Nodes_data_Target['Id']=Nodes_data_Target['Id'].str.lower()
frames = [Nodes_data_source, Nodes_data_Target]
Nodes_frame = pd.concat(frames)

Previa do Grafo que será gerado a partir dos dados extraídos, através da ferramenta Python. Serve para dar uma visão geral do que será melhor detalhado na ferramenta Gephi.

In [ ]:
Grafo = nx.from_pandas_edgelist(
    data4,
    source="Source",
    target="Target",
    create_using=nx.DiGraph
)

print(nx.info(Grafo))

grafo_nodes = Grafo.order()
grafo_edges = Grafo.size()
grafo_avg_deg = float(grafo_edges) / grafo_nodes
grafo_componentes_forte_conectados = nx.number_strongly_connected_components(Grafo)
grafo_componentes_fraco_conectados = nx.number_weakly_connected_components(Grafo)
grafo_indegrees = Grafo.in_degree()

print("Nodes:",grafo_nodes)
print("Edges:",grafo_edges)
print("Grau Médio:", grafo_avg_deg)
print("Qtde componentes com fraca conexao:", grafo_componentes_fraco_conectados)
print("Qtde componentes com forte conexao:",grafo_componentes_forte_conectados)

nnodes = Grafo.number_of_nodes()
degrees_in = [d for n, d in Grafo.in_degree()]
degrees_out = [d for n, d in Grafo.out_degree()]
avrg_degree_in = sum(degrees_in) / float(nnodes)
avrg_degree_out = sum(degrees_out) / float(nnodes)

in_values = sorted(set(degrees_in))
in_hist = [degrees_in.count(x) for x in in_values]
out_values = sorted(set(degrees_out))
out_hist = [degrees_out.count(x) for x in out_values]

plt.figure()
plt.loglog(in_values,in_hist)
plt.loglog(out_values,out_hist)
plt.legend(['In-degree','Out-degree'])
plt.xlabel('Grau')
plt.ylabel('Nro de Nodes')
plt.title('Grafo')
plt.show()
plt.close()

Lista as contas que mais foram citadas nos tweets (inbound)

In [ ]:
Liata_maiores_indegrees = sorted(Grafo.in_degree, key=lambda x: x[1], reverse=True)
res_list_in = [x[0] for x in Liata_maiores_indegrees]
res_list_in[:100]

Lista as contas que mais enviaram tweets (outbound)

In [ ]:
Liata_maiores_outdegrees = sorted(Grafo.out_degree, key=lambda x: x[1], reverse=True)
res_list_out = [x[0] for x in Liata_maiores_outdegrees]
users_maiores_outdegrees = client.get_users(ids=res_list_out[:100], user_fields=['id', 'username'])
users_maiores_outdegrees[:100]

Faz o merge com o arquivo de contas extraido a partir das listas de distribuição sobre as eleições, divulgadas pelo Twitter em 2022, onde foram extraídas os IDs, Nomes das contas, categoria de acordo com o nome da lista que a conta se encontrava (ex: Reporteres, Partidos Políticos, etc) e a declaração de voto de cada conta. A declaração se baseou na livre declaração de apoio aos candidatos Luis Inácio Lula da Silva e Jair Messias Bolsonaro. Caso essa declaração não tenha sido feita a conta era considerada como Neutra/Indefinida. 

In [ ]:
data6 = pd.DataFrame(users_maiores_outdegrees.data, columns = ["id", "name", "username"])
data6.rename(columns = {'id':'Id'}, inplace = True)
Nodes_frame_final = pd.merge(Nodes_frame, data6, how = 'left', on = 'Id')
Nodes_frame_final = Nodes_frame_final.reset_index(drop=True)
arq_contas_brasil = 'twitter_contas_br_2022_v3.csv'
df_contas_brasil = pd.read_csv(arq_contas_brasil, sep=';', low_memory = False, index_col=False)
df_contas_brasil['Id']=df_contas_brasil['Id'].str.lower()
Nodes_frame_final_categoria = pd.merge(Nodes_frame_final, df_contas_brasil, how='left', on=['Id'])

Geração do arquivo de Nodes para importação no Gephi

In [64]:
Nodes_frame_final_categoria.to_csv(file_nodes, index=False)


Geração de uma da Nuvem de Palavras com os tweets que foram coletados pela query 


In [68]:

Lista_palavras = ["vai","agora","HTTPS","Se", "O", "ao","de","a","o","que","e","do","da","em","um","para","é","com","não","uma","os","no","se","na","por","mais","as","dos","como","mas","foi","ao","ele","das","tem","à","seu","sua","ou","ser","quando","muito","há","nos","já","está","eu","também","só","pelo","pela","até","isso","ela","entre","era","depois","sem","mesmo","aos","ter","seus","quem","nas","me","esse","eles","estão","você","tinha","foram","essa","num","nem","suas","meu","às","minha","têm","numa","pelos","elas","havia","seja","qual","será","nós","tenho","lhe","deles","essas","esses","pelas","este","fosse","dele","tu","te","vocês","vos","lhes","meus","minhas","teu","tua","teus","tuas","nosso","nossa","nossos","nossas","dela","delas","esta","estes","estas","aquele","aquela","aqueles","aquelas","isto","aquilo","estou","está","estamos","estão","estive","esteve","estivemos","estiveram","estava","estávamos","estavam","estivera","estivéramos","esteja","estejamos","estejam","estivesse","estivéssemos","estivessem","estiver","estivermos","estiverem","hei","há","havemos","hão","houve","houvemos","houveram","houvera","houvéramos","haja","hajamos","hajam","houvesse","houvéssemos","houvessem","houver","houvermos","houverem","houverei","houverá","houveremos","houverão","houveria","houveríamos","houveriam","sou","somos","são","era","éramos","eram","fui","foi","fomos","foram","fora","fôramos","seja","sejamos","sejam","fosse","fôssemos","fossem","for","formos","forem","serei","será","seremos","serão","seria","seríamos","seriam","tenho","tem","temos","tém","tinha","tínhamos","tinham","tive","teve","tivemos","tiveram","tivera","tivéramos","tenha","tenhamos","tenham","tivesse","tivéssemos","tivesse","tiver","tivermos","tiverem","terei","terá","teremos","terão","teria","teríamos","teriam","Se","RT","se","ao"]

text = " ".join(i for i in data3.text)

stop_words = Lista_palavras + list(STOPWORDS)

regex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)

text_limpo_a = re.sub(regex_pattern,'',text) #replaces pattern with ''

re_list = ['@[A-Za-z0–9_]+', '#']
combined_re = re.compile( '|'.join( re_list) )
text_limpo_b = re.sub(combined_re,'',text_limpo_a)


wordcloud = WordCloud(max_words=50, collocation_threshold = 10, width = 1500, height = 1000, random_state = 42, collocations = False, colormap = 'tab10', stopwords = stop_words, background_color="white").generate(text_limpo_b)

plt.figure( figsize=(15,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

plt.tight_layout()

plt.show()